In [49]:
import pandas as pd

sf = "0.01"
input_type = "update_1.0"

# read the mysql query result
mysql_csv_path = f"mysql_test/ops_sf{sf}_{input_type}_mysql.csv"
mysql_df = pd.read_csv(mysql_csv_path)

# read the flink query result (in csv format)
flink_csv_path = f"output/ops_sf{sf}_{input_type}_output.csv"
flink_df = pd.read_csv(flink_csv_path, header=None, names=["c_custkey", "c_name", "c_acctbal", "c_address", "n_name", "c_phone", "c_comment", "deltaRevenue", "revenue"], delimiter="|")

# join the two dataframes
join_df = pd.merge(mysql_df, flink_df, on="c_custkey", how="outer", suffixes=("_mysql", "_flink"))

# find the consistent records
join_df["diff"] = join_df["revenue_mysql"] - join_df["revenue_flink"]
consistent = join_df[join_df["diff"] < 10e-6]
consistent_custkey = consistent["c_custkey"].unique()

all_custkey = mysql_df["c_custkey"].unique()
inconsistent_custkey = [int(custkey) for custkey in all_custkey if custkey not in consistent_custkey]

# calculate ratio of consistent and inconsistent
total_custkey = len(all_custkey)
consistent_ratio = len(consistent_custkey) / total_custkey
inconsistent_ratio = len(inconsistent_custkey) / total_custkey

print("consistent ratio: ", consistent_ratio)
print("inconsistent ratio: ", inconsistent_ratio)


consistent ratio:  1.0
inconsistent ratio:  0.0
